# Analyse and extract tables from pdf  

This notebook is designed to extract tables from PDF documents and save them as CSV files. It demonstrates the process of reading PDF files, extracting tabular data, and exporting the results for further analysis.

In [1]:
import pymupdf
from pprint import pprint
file_path = "../data/H129-108-2021-eng.pdf"
save_corrected = "../data/corrected_file.pdf"

In [ ]:
# check for page rotation
doc = pymupdf.open(file_path)
for i, page in enumerate(doc): # iterate the document pages
    tabs = page.find_tables()
    rotation = page.rotation  
    print(f"Page {i+1} rotation: {rotation} degrees")

In [3]:
import fitz
doc = fitz.open(file_path)

def detect_block_orientation(block, min_text_len=5, vertical_ratio=3):

    x0, y0, x1, y1, text, *_ = block
    text = text.strip()
    if len(text) <= min_text_len:
        return None
    
    width, height = (x1 - x0), (y1 - y0)
    if height > width * vertical_ratio:
        return "vertical"
    return "horizontal"


def analyze_page_orientation(page):
    blocks = page.get_text("blocks")
    orientations = []

    for b in blocks:
        orientations.append(detect_block_orientation(b))

    vcount = orientations.count("vertical")
    hcount = orientations.count("horizontal")
    # Decide majority orientation
    if vcount > hcount:
        return "vertical"
    elif hcount > vcount:
        return "horizontal"
    else:
        return "mixed"

fix_rotation = True

results = {}
for page_num, page in enumerate(doc):
    orientation = analyze_page_orientation(page)
    results[page_num] = orientation
    if fix_rotation and orientation == "vertical":
            page.set_rotation(90)
    
doc.save(save_corrected)
print(results)


{0: 'horizontal', 1: 'horizontal', 2: 'horizontal', 3: 'horizontal', 4: 'horizontal', 5: 'horizontal', 6: 'horizontal', 7: 'horizontal', 8: 'horizontal', 9: 'horizontal', 10: 'horizontal', 11: 'horizontal', 12: 'horizontal', 13: 'horizontal', 14: 'horizontal', 15: 'horizontal', 16: 'horizontal', 17: 'horizontal', 18: 'horizontal', 19: 'horizontal', 20: 'horizontal', 21: 'horizontal', 22: 'horizontal', 23: 'vertical', 24: 'vertical', 25: 'vertical', 26: 'vertical', 27: 'vertical', 28: 'vertical', 29: 'vertical', 30: 'vertical', 31: 'vertical', 32: 'vertical', 33: 'vertical', 34: 'vertical', 35: 'vertical', 36: 'vertical', 37: 'vertical', 38: 'vertical', 39: 'vertical', 40: 'vertical', 41: 'vertical', 42: 'vertical', 43: 'vertical', 44: 'vertical', 45: 'vertical', 46: 'vertical', 47: 'vertical', 48: 'vertical', 49: 'vertical', 50: 'vertical', 51: 'vertical', 52: 'vertical', 53: 'vertical', 54: 'vertical', 55: 'vertical', 56: 'horizontal', 57: 'horizontal', 58: 'horizontal', 59: 'horizont

In [4]:
import fitz  # PyMuPDF
import pandas as pd


doc = fitz.open(save_corrected)

for i, page in enumerate(doc):
    tabs = page.find_tables()
    for table in tabs.tables:
        df = pd.DataFrame(table.extract())
        if i+1 in [16, 17, 18] and len(df.columns) == 5:
            df.iat[0, 2] = df.iat[0, 1]
            df.iat[0, 4] = df.iat[0, 3]
            merged_header = df.iloc[0].astype(str) + "_" + df.iloc[1].astype(str)
            df.columns = merged_header
            df = df.drop([0, 1])
        elif i+1 in [16, 17, 18] and len(df.columns) == 6:
            df.iat[0, 3] = df.iat[0, 2]
            df.iat[0, 5] = df.iat[0, 4]
            merged_header = df.iloc[0].astype(str) + "_" + df.iloc[1].astype(str)
            df.columns = merged_header
            df = df.drop([0, 1])
        
        else:
            df.columns = df.iloc[0]   # take row 0 as column names
            df = df.drop(df.index[0]) # drop row 0 from data
            df = df.reset_index(drop=True)
        
        if i+1 == 21:
            left  = pd.concat([df.iloc[:, 0], df.iloc[:, 2]], axis=0, ignore_index=True)
            # col 1 + col 3
            right = pd.concat([df.iloc[:, 1], df.iloc[:, 3]], axis=0, ignore_index=True)
            df = pd.DataFrame({"Substance": left, "RAF Derm": right})
        
        df.iloc[:, 0] = df.iloc[:, 0].ffill()
        df.to_csv(f"../data/output_table{i+1}.csv", index=False)

        #df.columns = [f"{col}_{i}" if df.columns.tolist().count(col) > 1 else col 
        #      for i, col in enumerate(df.columns)]
        
        display(  # <-- Jupyter rich display
                    df.style.set_table_styles(
                    [
                        {"selector": "thead th", "props": [("font-weight", "bold"), ("background-color", "#dbeafe"), ("color", "black")]},
                        {"selector": "tbody td", "props": [("border", "1px solid #ccc"), ("padding", "6px")]}
                    ]
                ).set_properties(**{"text-align": "center"})
                 .set_caption(f"📄 Table from Page {i+1}")
            )

,Substance_None,Non-Carcinogenic TRVs*_Oral Tolerable Daily Intake TDI mg/kg -day BW,Non-Carcinogenic TRVs*_Inhalation Tolerable Concentration TC mg/m3,Carcinogenic TRVs*_Oral Slope Factor SF (mg/kg -day)-1 BW,Carcinogenic TRVs*_Inhalation Unit Risk UR (mg/m3)-1
2,Arsenic,,,1.8,6.4
3,Barium,0.2,,,
4,Benzene,,,0.083,0.016
5,Benzo[a]pyrene (B[a]P),0.0000667,0.000002,1.289,0.6
6,Beryllium,0.002,0.00002,,2.4
7,Cadmium,0.0008p,,,4.2
8,Carbon tetrachloride1,0.00071,,,0.006
9,Chlorobenzene,0.43,0.01p,,
10,"Chromium, trivalent",1.5,0.0001,,
11,"Chromium, hexavalent",0.0022,0.0001,,76


,Substance_None,None_None,Non-Carcinogenic TRVs*_Oral Tolerable Daily Intake TDI mg/kg -day BW,Non-Carcinogenic TRVs*_Inhalation Tolerable Concentration TC mg/m3,Carcinogenic TRVs*_Oral Slope Factor SF (mg/kg -day)-1 BW,Carcinogenic TRVs*_Inhalation Unit Risk UR (mg/m3)-1
2,Ethylbenzene,None,0.022,2,,
3,n-Hexane,None,0.1p,0.7p,,
4,Lead3,None,0.0005p,,,
5,Manganese,None,0.025,,,
6,"Mercury, inorganic4",None,0.0003,,,
7,"Methylmercury women of child-bearing age, infants and children < 12 years non-sensitive adults of the general population",None,0.0002p 0.00047p,,,
8,"Methylnaphthalene, 2-",None,0.004,,,
9,Naphthalene,None,0.02,0.01,,
10,Nickel chloride,None,0.0013,,,
11,Nickel oxide,None,,0.000025,,


,Substance_None,None_None,Non-Carcinogenic TRVs*_Oral Tolerable Daily Intake TDI mg/kg -day BW,Non-Carcinogenic TRVs*_Inhalation Tolerable Concentration TC mg/m3,Carcinogenic TRVs*_Oral Slope Factor SF (mg/kg -day)-1 BW,Carcinogenic TRVs*_Inhalation Unit Risk UR (mg/m3)-1
2,Zinc,0 to < 6 months,0.49UL,,,
3,Zinc,6 months to < 5 years,0.48UL,,,
4,Zinc,5 to < 12 years,0.51UL,,,
5,Zinc,12 to < 20 years,0.54UL,,,
6,Zinc,≥ 20 years,0.57UL,,,


0,PAH,CAS No.,Benzo[a]Pyrene RPF1
0,Benzo[a]pyrene,50-32-8,1
1,Benzo[a]anthracene,56-55-3,0.1
2,Benzo[b]fluoranthene,205-99-2,0.1
3,"Benzo[g,h,i]perylene",191-24-2,0.01
4,Benzo[j]fluoranthene,205-82-3,0.1
5,Benzo[k]fluoranthene,207-08-9,0.1
6,Chrysene,218-01-9,0.01
7,"Dibenzo[a,h]anthracene",53-70-3,1
8,"Indeno[1,2,3-cd]pyrene",193-39-5,0.1


0,PAH,CAS No.,Provisional Benzo[a]pyrene RPF1
0,Anthanthrene,191-26-4,0.1
1,Benzo[c]chrysene,194-69-4,0.01
2,Benzo[g]chrysene,196-78-1,0.1
3,Benzo[c]phenanthrene,195-19-7,0.01
4,"Cyclopenta[c,d]pyrene",27208-37-3,0.1
5,"Dibenzo[a,e]fluoranthene",5385-75-1,1
6,"Dibenzo[a,e]pyrene",192-65-4,1
7,"Dibenzo[a,h]pyrene",189-64-0,1
8,"Dibenzo[a,i]pyrene",189-55-9,1
9,"Dibenzo[a,l]pyrene",191-30-0,100


...

0,Substance,Type of TRV,TRV Value,Study Details,Threshold/ Non-threshold Endpoint,TRV Derivation Method,Critical Effect(s),Carcinogenicity Classification,Source
0,Trichloroethylene (TCE),Inhalation TC,2.0E-03 mg/m3,"Keil et al., 2009","LOAEL = 0.35 mg/kg -day BW POD (LOAEL) internal dose = 0.139 mg TCE metabolized / kg 3/4/day BW HEC = 99, LOAEL 0.033 ppm (0.19 mg/m3)","Candidate RfCs derived using a PBPK model integrating combined intraspecies, interspecies, and route-to-route extrapolation, and dividing by a UF. Candidate RfC (Keil et al., 2009) = 0.0019 mg/m3 Candidate RfC (Johnson et al., 2003) = 0.0021 mg/m3 Selected RfC = midpoint between the candidate RfCs = 0.002 mg/m3",Developmental toxicity (fetal heart malformations) and Immunotoxicity (decreased thymus weight),"CEPA: Group II probably carcinogenic to humans (EC and HC, 1993g) IARC: Group 1 carcinogenic to humans (IARC, 2014) US EPA IRIS: carcinogenic to humans (US EPA, 2011b)","US EPA, 2011b (based on Keil et al., 2009 and Johnson et al., 2003)"
1,Trichloroethylene (TCE),None,None,"Study Type: chronic Species: female B6C3F1 mice Mode of Administration: oral (drinking water) Exposure Regime: 0, 1.4, and 14 ppm (0, 0.35, 3.5 mg/kg -day) BW Duration: 30 weeks Uncertainty Factors: 100 (10 for extrapolating from a LOAEL rather than a NOAEL, 3 for intraspecies variability, and 3 for interspecies variability) [total uncertainty factor rounded to 100]",None,None,None,None,None
2,Trichloroethylene (TCE),None,None,"Johnson et al., 2003","POD = BMDL internal dose 01 = 0.0142 mg TCE metabolized by oxidation/kg 3/4/day BW HEC = 99, BMDL01 0.0037 ppm (0.021 mg/m3)",None,None,None,None
3,Trichloroethylene (TCE),None,None,"Study Type: developmental Species: pregnant Sprague-Dawley rats Mode of Administration: oral (drinking water) Exposure Regime: 0, 0.0025, 0.25, 1.5, and 1100 ppm (0, 0.00045, 0.048, 0.218 or 129 mg/kg -day) on BW gestational days 1 to 22 Duration: 3 weeks during pregnancy Uncertainty Factors: 10 (3 for intraspecies variability, and 3 for interspecies variability) [total uncertainty factor rounded to 10]",None,None,None,None,None
4,Trichloroethylene (TCE),Oral SF,8.11E-04 (mg/kg -day)–1 BW,"Study Type: chronic Species: male and female rats Mode of Administration: gavage Exposure Regime: 0, 500, and 1000 mg/kg -day, 5 days/week BW Duration: 103 weeks Uncertainty Factors: N/A",Range of oral SFs: 5.82E-04 to 8.11E-04 (mg/kg -day)–1 BW,Linearized multistage model and allometric scaling Most conservative oral SF: 8.11E-04 (mg/kg -day)–1 BW,Cancer (kidney [combined tubular cell adenomas and adenocarcinomas]),None,"HC, 2005 (based on NTP, 1988 and NTP, 1990)"


0,Substance,Type of TRV,TRV Value,Study Details,Threshold/ Non-threshold Endpoint,TRV Derivation Method,Critical Effect(s),Carcinogenicity Classification,Source
0,Trichloroethylene (TCE),Inhalation UR,4.1E-03 (mg/m3)–¹,Study Type: epidemiological (occupational) Species: humans Mode of Exposure: inhalation Exposure Concentrations: N/A Duration: chronic Uncertainty Factors: N/A,LEC 01 = 2.4 mg/m3 (lowest effective concentration),Linear low-dose extrapolation from the LEC (95% lower bound on 01 the exposure associated with a 1% extra cancer risk) + Application of a factor of 4 to include non-Hodgkin’s lymphoma and liver cancer risks Inhalation UR = 0.01 / LEC 01 where 0.01 = 1% extra cancer risk,"Cancer (liver, kidney [renal cell carcinoma], non-Hodgkin’s lymphoma)","CEPA: Group II probably carcinogenic to humans (EC and HC, 1993g) IARC: Group 1 carcinogenic to humans (IARC, 2014) US EPA IRIS: carcinogenic to humans (US EPA, 2011b)","US EPA, 2011b (based on Charbotel et al., 2006 and Raaschou-Nielsen et al., 2003)"
1,Uranium (non-radioactive),Oral TDI,6.0E-04 mg/kg -day BW,"Study Type: subchronic Species: male and female Sprague-Dawley rats Mode of Administration: oral (drinking water) Exposure Regime: 0, 0.96, 4.8, 24, 120, or 600 mg/L uranyl nitrate hexahydrate (equivalent to uranium doses of 0, 0.06, 0.31, 1.52, 7.54, and 36.73 mg/kg -day BW in male rats, and 0, 0.09, 0.42, 2.01, 9.98, and 53.56 mg/kg -day in female rats) BW Duration: 91 days Uncertainty Factors: 100 (10 for intraspecies variability and 10 for interspecies variability). Other points of departure (including a NOAEL) were higher than the LOAEL identified by Gilman et al. (1998) and therefore HC (2019c) did not consider an uncertainty factor for use of a LOAEL instead of a NOAEL necessary. HC (2019c) did not apply an uncertainty factor for use of a subchronic study because the study was considered adequately sensitive since the observed kidney effects were similar to the minimal effects seen in a number of longer-term studies and since human absorption values were found to be independent of exposure duration (HC, 2019c).",LOAEL = 0.06 mg/kg -day BW (males),TDI = LOAEL/UF,Nephrotoxicity (renal lesions),CEPA: not assessed IARC: not assessed US EPA IRIS: not assessed,"HC, 2019c (based on Gilman et al., 1998)"


0,Substance,Type of TRV,TRV Value,Study Details,Threshold/ Non-threshold Endpoint,TRV Derivation Method,Critical Effect(s),Carcinogenicity Classification,Source
0,Vinyl chloride,Oral SF,2.4E-01 (mg/kg -day)–¹ BW for continuous lifetime exposure during adulthood 4.8E-01 (mg/kg -day)–¹ BW for continuous lifetime exposure from birth,"Study Type: chronic Species: male and female rats Mode of Administration: diet (mixture of vinyl chloride monomer [VCM] and polyvinyl chloride [PVC] powder) Exposure Regime: 0, 1.7, 5.0, or 14.1 mg/kg -day, 4 hour feeding BW period/day; a positive control group was administered 300 mg/kg VCM BW in soybean oil by stomach tube, 5 days/week Duration: lifetime (the experiment was terminated once 75% mortality was observed in the positive control group, i.e. 135 weeks for males and 144 weeks for females) Uncertainty Factors: N/A",External human dose associated with an excess lifetime risk of 10-5 for combined liver cancers = 4.19E-05 mg/kg -day BW,"Rat PBPK model to determine daily internal doses of vinyl chloride liver metabolites + Multistage model to determine a POD + Human PBPK model to estimate external doses Oral SF = 10-5 / external human dose Given animal evidence of early-life sensitivity to vinyl chloride, a factor of 2 was applied to the oral SF of 2.4E-01 (mg/kg -day)–¹ BW for exposure during adulthood, to account for continuous lifetime exposure from birth (HC, 2013c).",Cancer (liver [hepatocellular angiosarcomas and carcinomas]),"CEPA: known human carcinogen (EC and HC, 2016) IARC: Group 1 carcinogenic to humans (IARC, 2012b) US EPA IRIS: Group A carcinogenic to humans (US EPA, 2000b)","HC, 2013c (based on Feron et al., 1981)"
1,Vinyl chloride,Inhalation UR [US EPA (2000b): should not be used for exposures >10 mg/m3],4.4E-03 (mg/m3)–1 for continuous lifetime exposure during adulthood 8.8E-03 (mg/m3)–1 for continuous lifetime exposure from birth,"Study Type: chronic Species: Sprague-Dawley female rats Mode of Administration: inhalation (whole body exposure chambers) Exposure Regime: 0, 1, 5, 10, 25, 50, 100, 150, 200, 250, 500, 2500, 6000, or 10 000 ppm (0, 2.6, 12.8, 25.6, 63.9, 128, 256, 383, 511, 639, 1278, 6390, 15 340, 25 560 mg/m3) vinyl chloride, 4 hours/day, 5 days/week Duration: 52 weeks Uncertainty Factors: N/A A two-fold safety factor was applied to the adult value to account for continuous lifetime exposure from birth.",Based on the 95% upper confidence limit on excess cancer risk in female rats,PBPK model and linearized multistage model,"Cancer (liver [angiosarcomas, angiomas, hepatomas, and neoplastic nodules])",None,"US EPA, 2000b (based on Maltoni et al., 1981 and 1984)"


0,Substance,Type of TRV,TRV Value,Study Details,Threshold/ Non-threshold Endpoint,TRV Derivation Method,Critical Effect(s),Carcinogenicity Classification,Source
0,"Xylenes, mixed isomers",Oral TDI,1.3E-02 mg/kg -day BW,"Study Type: subchronic Species: male Wistar rats Mode of Administration: inhalation (whole body exposure chambers) Exposure Regime: a control group and 3 exposure groups: 1) m-xylene concentrations of 50 ppm (217 mg/m3) and 100 ppm (435 mg/m3), or 2) n-butyl alcohol 50 ppm (154 mg/m3) and 100 ppm (308 mg/m3), or 3) a 1:1 mixture of m-xylene and n-butyl alcohol (100 ppm [217 mg/m3 m-xylene + 154 mg/m3 of n-butyl alcohol] and 200 ppm [435 mg/m3 m-xylene + 308 mg/m3 of n-butyl alcohol]), 6 hours/day, 5 days/week Duration: 3 months Uncertainty Factors: 75 (10 for intraspecies variability, 2.5 for interspecies variability, and 3 for use of a subchronic study)",NOAEL = 50 ppm (217 mg/m3) m-xylene,PBPK modeling to estimate internal rat blood concentration corresponding to NOAEL of 50 ppm = 0.138 mg/L + Conversion to external oral human dose using human PBPK model and assuming ingestion of 1.5 L drinking water/day NOAEL = HEC 1.0 mg/kg -day BW TDI = NOAEL /UF HEC,Neurotoxicity (impaired motor coordination),"CEPA: Group IV unlikely to be carcinogenic to humans (EC and HC, 1993h) IARC: Group 3 not classifiable as to its carcinogenicity to humans (IARC, 1999a) US EPA IRIS: inadequate information to assess carcinogenic potential (US EPA, 2003b)","HC, 2014b (based on Korsak et al., 1994)"
1,"Xylenes, mixed isomers",Inhalation TC,1.0E-01 mg/m3,"Study Type: subchronic Species: male Wistar rats Mode of Administration: inhalation (whole body exposure chambers) Exposure Regime: a control group and 3 exposure groups: 1) m-xylene concentrations of 50 ppm (217 mg/m3) and 100 ppm (435 mg/m3), or 2) n-butyl alcohol 50 ppm (154 mg/m3) and 100 ppm (308 mg/m3), or 3) a 1:1 mixture of m-xylene and n-butyl alcohol (100 ppm [217 mg/m3 m-xylene + 154 mg/m3 of n-butyl alcohol] and 200 ppm [435 mg/m3 m-xylene + 308 mg/m3 of n-butyl alcohol]), 6 hours/day, 5 days/week Duration: 3 months Uncertainty Factors: 300 (10 for intraspecies human variability, 3 for interspecies variability, 3 for extrapolation from subchronic to chronic duration, and 3 for database uncertainties)",NOAEL = 50 ppm (217 mg/m3),NOAEL adjusted for continuous exposure and difference in blood/gas partitioning in rats vs humans NOAEL = 39 mg/m3 HEC TC = NOAEL /UF HEC,Neurotoxicity (impaired motor coordination),None,"HC, 2018a (based on US EPA, 2003b [derived from Korsak et al., 1994])"


0,Substance,Type of TRV,TRV Value,Study Details,Threshold/ Non-threshold Endpoint,TRV Derivation Method,Critical Effect(s),Carcinogenicity Classification,Source
0,Zinc,UL (HC) __________ 0 to <6 mo 6 mo to <5 yrs 5 to <12 yrs 12 to <20 yrs ≥20 years,mg/kg -day BW __________ 4.9E-01 4.8E-01 5.1E-01 5.4E-01 5.7E-01,"Yadrick et al., 1989 (adults)",LOAEL = 60 mg/day (adults),"UL (IOM) = LOAEL/UF for intake of zinc from food, water, and supplements IOM adult ULs were adjusted to account for differences in HC’s adult age group (HC, 2010)","Decrease in erythrocyte superoxide dismutase (ESOD) activity (sensitive indicator of copper status, reflecting copper utilization and the risk of copper deficiency)","CEPA: see 2019 (draft) CMP assessment (ECCC and HC, 2019c) IARC: not assessed US EPA IRIS: Group D not classifiable as to human carcinogenicity (US EPA, 2005d)","IOM, 2001 (based on Yadrick et al., 1989, and Walravens and Hambidge, 1976)"
1,Zinc,None,None,Study Type: epidemiological (prospective) Species: humans (adult women) Mode of Exposure: dietary supplements Exposure Concentrations: 10 mg/day (estimated dietary intake) + supplemental intake of 50 mg/day as zinc gluconate Duration: 10 weeks Uncertainty Factors: 1.5 (for intraspecies variability and extrapolation from a LOAEL to a NOAEL),None,None,None,None,None
2,Zinc,None,None,"Walravens and Hambidge, 1976 (infants, children, and adolescents)",NOAEL = 5.8 mg/L (infants),"NOAEL adjusted for estimated average human milk intake of 0.78 L/day NOAEL = 4.5 mg/day adj Infant UL (IOM) = NOAEL /UF = adj 4.5 mg/day (4 mg/day rounded down) IOM derived ULs for older infants, children, and adolescents based on the infant UL and relative body weight IOM ULs were adjusted to account for differences in HC’s age groups (HC, 2010)",No effects of zinc on serum copper or cholesterol concentrations or other adverse effects were found,None,None
3,Zinc,None,None,"Study Type: epidemiological (prospective) Species: humans (infants, 0-6 months) Mode of Exposure: dietary supplements Exposure Concentrations: control group: formula with 1.8 mg zinc/L; exposure group: formula with 1.8 mgzinc/L + supplement with 4 mg zinc/L (total of 5.8 mg zinc/L) Duration: 6 months Uncertainty Factors: 1 (because of a lack of evidence that formula intakes of 5.8 mg zinc/L result in infant toxicity)",None,None,None,None,None
